In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 

In [2]:
dataset_path = os.listdir('room_dataset')
room_types = os.listdir('room_dataset')

print(room_types)
print("Types of rooms found: ", len(room_types))

['bed_room', 'dining_room', 'living_room']
Types of rooms found:  3


In [3]:
rooms = []

for item in room_types:
    all_rooms = os.listdir('room_dataset' + '/' + item)

    for room in all_rooms:
        rooms.append((item, str('room_dataset' + '/' + item) + '/' + room))
        print(rooms[-1])


('bed_room', 'room_dataset/bed_room/bed-1303451__340.jpg')
('bed_room', 'room_dataset/bed_room/bed-1846251__340.jpg')
('bed_room', 'room_dataset/bed_room/bed-3786264__340.jpg')
('bed_room', 'room_dataset/bed_room/bed-4065946__340.jpg')
('bed_room', 'room_dataset/bed_room/bed-4343379__340.jpg')
('bed_room', 'room_dataset/bed_room/bed-4343382__340.jpg')
('bed_room', 'room_dataset/bed_room/bed-4540208__340.jpg')
('bed_room', 'room_dataset/bed_room/bedroom-3102376__340.jpg')
('bed_room', 'room_dataset/bed_room/bedroom-374982__340.jpg')
('bed_room', 'room_dataset/bed_room/bedroom-4072391__340 (1).jpg')
('bed_room', 'room_dataset/bed_room/bedroom-460762__340.jpg')
('bed_room', 'room_dataset/bed_room/bedroom-690129__340.jpg')
('dining_room', 'room_dataset/dining_room/apartment-185778__340.jpg')
('dining_room', 'room_dataset/dining_room/apartment-185779__340.jpg')
('dining_room', 'room_dataset/dining_room/apartment-2094648__340.jpg')
('dining_room', 'room_dataset/dining_room/architectural-2242

In [4]:
rooms_df = pd.DataFrame(data=rooms, columns=['room type', 'image'])
print(rooms_df.head())

  room type                                       image
0  bed_room  room_dataset/bed_room/bed-1303451__340.jpg
1  bed_room  room_dataset/bed_room/bed-1846251__340.jpg
2  bed_room  room_dataset/bed_room/bed-3786264__340.jpg
3  bed_room  room_dataset/bed_room/bed-4065946__340.jpg
4  bed_room  room_dataset/bed_room/bed-4343379__340.jpg


In [5]:
print("Total number of rooms in dataset: ", len(rooms_df))

room_count = rooms_df['room type'].value_counts()

print("rooms in each categpry: ")
print(room_count)

Total number of rooms in dataset:  47
rooms in each categpry: 
room type
living_room    24
bed_room       12
dining_room    11
Name: count, dtype: int64


In [6]:
import cv2
path = 'room_dataset/'

im_size = 244

images = []
labels = []

for i in room_types:
    data_path = path + str(i)
    filenames = [i in os.listdir(data_path)]

    for file in filenames:
        img = cv2.imread(data_path + '/' + file)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

TypeError: can only concatenate str (not "bool") to str

In [ ]:
images = np.array(images)

images = images / 255.0
images.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

y = rooms_df['room type'].vales

y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(y)

In [ ]:
y = y.reshape(-1, 1)
onehotencoder = OneHotEncoder(categorical_features=[0])
Y = onehotencoder.fit_transform(y)
Y.shape

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(images, Y, test_size = 0.1, random_state=42)

print(X_train.shaep, X_test.shape)
print(y_train.shaep, y_test.shape)


In [ ]:
import numpy as np 
from keras import layers 
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model 
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

In [ ]:
def identity_block(x, f, filters):
    F1, F2, F3 = filters #64, 64, 256
    x_shortcut = x 

    # First layer 
    x = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1, 1), padding = 'valid')(x)
    x = BatchNormalization(axis = 3)(x)
    x = Activation('relu')(x)

    # Second layer
    x = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same')(x)
    x = BatchNormalization(axis = 3)(x)
    x = Activation('relu')(x)

    # Third layer
    x = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid')(x)
    x = BatchNormalization(axis = 3)(x)


    # Final step: Add shortcut value to F(x), and pass it through a RELU activation
    x = Add([x, x_shortcut])
    x = Activation('relu')(x)

    return x

In [ ]:
def convalutional_block(x, f, filters, s=2):
    F1, F2, F3 = filters #64, 64, 256
    x_shortcut = x 

    # First layer 
    x = Conv2D(filters = F1, kernel_size = (1, 1), strides = (s, s))(x)
    x = BatchNormalization(axis = 3)(x)
    x = Activation('relu')(x)

    # Second layer
    x = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same')(x)
    x = BatchNormalization(axis = 3)(x)
    x = Activation('relu')(x)

    # Third layer
    x = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid')(x)
    x = BatchNormalization(axis = 3)(x)

    ### Shorcut Path
    x_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s, s), padding = 'valid')(x_shortcut)
    x_shortcut = BatchNormalization(axis = 3)(x_shortcut)

    # Final step: Add shortcut value to F(x), and pass it through a RELU activation
    x = Add([x, x_shortcut])
    x = Activation('relu')(x)

    return x

In [ ]:
def ResNet50(input_shape=(224, 224, 3), classes=3):
    """ 
    Implementation of the ResNet-50 architecture:
    Conv2D -> Batch Norm -> Relu -> Max Pool -> Conv Block -> ID Block*2 -> Conv Block -> ID Block*3
    -> Conv Block -> ID Block*5 -> Conv Block -> ID Block*2 -> Avg Pool -> Top Player
    """

    # Define the input with the shape input_shape
    x_input = Input(input_shape)

    # Zero-padding 
    x = ZeroPadding2D((3, 3))(x_input) # 3,3 padding

    # Stage 1
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization(axis = 3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), stride=(2, 2))(x)

    # Stage 2
    x = convalutional_block(x, f=3, filters=[64, 64, 256], s=1)

    x = identity_block(x, 3, [64, 64, 256])

    x = identity_block(x, 3, [64, 64, 256])

    # stage 3
    x = convalutional_block(x, f=3, filters=[128, 128, 512], s=2)
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    x = identity_block(x, 3, [128, 128, 512])
    
    # stage 4
    x = convalutional_block(x, f=3, filters=[256, 256, 1024], s=2)
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])
    x = identity_block(x, 3, [256, 256, 1024])

    # stage 5
    x = convalutional_block(x, f=3, filters=[512, 512, 2048], s=2)
    x = identity_block(x, 3, [512, 512, 2048]) 
    x = identity_block(x, 3, [512, 512, 2048]) 

    # Avg Pool
    x = AveragePooling2D((2, 2), name='avg_pool')(x)

    # Output layer
    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc' + str(classes), keras_initializer = glorot_uniform(seed=0))(x)

    # Create model 
    model = Model(inputs = x_input, outputs = x, name='ResNet50')

    return model 



In [ ]:
model = ResNet50(input_shape = (224, 224, 3), classes = 3)

In [ ]:
model.compile(optimizer='adam', loss='catagorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs = 10, batch_size = 32)

In [ ]:
preds = model.evaluate((X_test, y_test))
print("Loss: ", str(preds[0]))
print("Task Accuracy: ", str(preds[1]))

In [ ]:
from matplotlib.pyplot import imread 

img_path = 'test_img.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print("Input image shape: ", x.shape)
my_image = imread(img_path)
imshow(my_image)
print(model.predict(x))